In [1]:
import pandas as pd
import numpy as np
import re
import sys

sys.path.append("..")
from src import parse_salary_row, extract_skills, soft_skills, TECHNICAL_SKILL_ALIASES, title_pattern, desc_pattern, exclude_pattern, build_alias_lookup, build_skill_regex

## Load in datasets (from multiple sources)

In [2]:
# https://www.kaggle.com/datasets/lukebarousse/data-analyst-job-postings-google-search
# taken from google search
df = pd.read_csv("../data/gsearch_jobs.csv")

# https://www.kaggle.com/datasets/rashikrahmanpritom/data-science-job-posting-on-glassdoor?select=Cleaned_DS_Jobs.csv
# all from glassdoor
df2 = pd.read_csv("../data/Cleaned_DS_Jobs_2020.csv")

# https://www.kaggle.com/datasets/elahehgolrokh/data-science-job-postings-with-salaries-2025
#df2 = pd.read_csv("data_science_job_posts_2025.csv")

### First dataframe cleaning

let's take this dataframe and extract entry-level/new-grad positions using key words found in either the job title or the description

In [3]:
# Query out non entry level jobs through text matching
df["is_entry_level"] = (
    df["title"].str.lower().str.contains(title_pattern, regex=True, na=False)
    |
    df["description"].str.lower().str.contains(desc_pattern, regex=True, na=False)
)

df["is_entry_level"] = df["is_entry_level"] & (
    ~df["title"].str.lower().str.contains(exclude_pattern, regex=True, na=False)
)

entry_jobs = df[df["is_entry_level"]].drop("is_entry_level",axis=1).reset_index(drop=True)

In [4]:
#extract the year
entry_jobs["date_time"] = pd.to_datetime(df["date_time"])
entry_jobs["year"] = entry_jobs["date_time"].dt.year

In [5]:
# columns we will be keeping
col_keep = ['title', "year","via","salary_pay","salary_rate","description","description_tokens","location","work_from_home"]

In [6]:
entry_jobs_cleaned = entry_jobs[col_keep].copy()

# via column = source of job post
# change NaN values to be False
entry_jobs_cleaned['via'] = entry_jobs_cleaned['via'].str.strip('via ')
entry_jobs_cleaned['work_from_home'] = entry_jobs_cleaned['work_from_home'].fillna(False)

C:\Users\antho\AppData\Local\Temp\ipykernel_13276\2168505758.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  entry_jobs_cleaned['work_from_home'] = entry_jobs_cleaned['work_from_home'].fillna(False)


In [7]:
#cleaning salary, works for both hourly, monthly, and yearly salary text
entry_jobs_cleaned[["salary_min_annual", "salary_max_annual", "salary_type"]] = df.apply(
    lambda r: parse_salary_row(r["salary_pay"], r["salary_rate"]),
    axis=1
)

In [8]:
entry_cleaned = entry_jobs_cleaned.drop(["salary_pay","salary_rate"],axis = 1)

In [9]:
entry_cleaned.head()

,title,year,via,description,description_tokens,location,work_from_home,salary_min_annual,salary_max_annual,salary_type
0,Data Analyst,2023,LinkedIn,Job Title: Entry Level Business Analyst / Prod...,[],United States,False,101000.0,143000.0,annual
1,Data Analyst - Consumer Goods - Contract to Hire,2023,Upwork,Enthusiastic Data Analyst for processing sales...,"['powerpoint', 'excel', 'power_bi']",Anywhere,True,NaN,NaN,NaN
2,Data Warehouse and Data Analyst,2023,LinkedIn,Be part of a team that unleashes the power of ...,['sql'],"Kansas City, KS",False,NaN,NaN,NaN
3,Associate Research/Data Analyst-CES - Now Hiring,2023,Snagajob,Why you'll love working for this Department:\n...,[],"Jefferson City, MO",False,31200.0,52000.0,hourly
4,"Data Analyst, Strategy",2023,Jobs Trabajo.org,Everything we do at Sunrun is driven by a dete...,"['tableau', 'snowflake', 'sql', 'python']","Jefferson City, MO",False,90000.0,110000.0,annual


Let's now extract skills from the description (both technical and soft). We already have a column "description_tokens" provided for us but let's see if we can get something more comprehensive before we drop it

In [10]:
# just checking what skills they included. Could be useful to add to my running list
# entry_cleaned.description_tokens
# unique_skills = entry_cleaned.description_tokens.apply(lambda x: x[1:-1].split(','))
# skills = set(
#     skill
#     for sublist in unique_skills
#     for skill in sublist
# )

# skills

In [14]:
alias_lookup = build_alias_lookup(TECHNICAL_SKILL_ALIASES)
skill_pattern = build_skill_regex(TECHNICAL_SKILL_ALIASES)

tech_skills = entry_cleaned["description"].apply(
    lambda txt: extract_skills(txt, alias_lookup, skill_pattern)
)

In [21]:
tech_skills.iloc[4710]#.iloc[5:20].loc[15]

['aws',
 'redshift',
 'sagemaker',
 'python',
 'scala',
 'r',
 'spark',
 'sql',
 'pandas',
 'numpy']

In [22]:
entry_cleaned.description_tokens.iloc[4710]

"['pandas', 'redshift', 'aws', 'python', 'numpy', 'spark', 'sql', 'scala']"

In [44]:
import ast

entry_cleaned["description_tokens"] = entry_cleaned["description_tokens"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [47]:
entry_cleaned["description_tokens"]

0                                                      []
1                           [powerpoint, excel, power_bi]
2                                                   [sql]
3                                                      []
4                       [tableau, snowflake, sql, python]
                              ...                        
4709                  [power_bi, tableau, sql, r, python]
4710    [pandas, redshift, aws, python, numpy, spark, ...
4711                                                [sql]
4712                                                [sql]
4713                                                [sql]
Name: description_tokens, Length: 4714, dtype: object

In [48]:
from collections import Counter

tech_skill_counts = Counter(
    skill
    for lst in tech_skills
    for skill in lst
)

token_skill_counts = Counter(
    skill
    for lst in entry_cleaned["description_tokens"].dropna()
    for skill in lst
)

In [58]:
tech_skills.to_csv('our_cleaned.csv', index=True, header=True)

In [56]:
tech_skill_counts

Counter({'sql': 2096,
         'excel': 1876,
         'python': 1516,
         'tableau': 1426,
         'power bi': 1325,
         'r': 972,
         'word': 360,
         'powerpoint': 352,
         'java': 309,
         'go': 265,
         'aws': 210,
         'javascript': 178,
         'azure': 172,
         'snowflake': 159,
         'spss': 137,
         'outlook': 134,
         'databricks': 117,
         'sharepoint': 110,
         'visual basic': 104,
         'docker': 99,
         'looker': 97,
         'spark': 95,
         'pyspark': 84,
         'tensorflow': 79,
         'pandas': 72,
         'mysql': 71,
         'nosql': 69,
         'scala': 65,
         'git': 64,
         'bigquery': 63,
         'redshift': 63,
         'hadoop': 61,
         'postgres': 58,
         'cognos': 58,
         'qlik': 57,
         'jira': 54,
         'numpy': 53,
         'airflow': 53,
         'linux': 51,
         'visio': 47,
         'gcp': 45,
         'shell': 43,
         '

In [54]:
token_skill_counts

Counter({'sql': 2048,
         'excel': 1792,
         'python': 1489,
         'tableau': 1402,
         'power_bi': 1357,
         'r': 853,
         'word': 348,
         'powerpoint': 342,
         'sas': 332,
         'java': 305,
         'c': 241,
         'spreadsheet': 206,
         'aws': 203,
         'go': 201,
         'azure': 162,
         'javascript': 160,
         'snowflake': 157,
         'c++': 144,
         'alteryx': 143,
         'sap': 138,
         'spss': 136,
         'outlook': 131,
         'sharepoint': 105,
         'docker': 99,
         'looker': 94,
         'spark': 90,
         'pyspark': 82,
         'vba': 80,
         'tensorflow': 74,
         'pandas': 72,
         'ssis': 69,
         'mysql': 68,
         'bigquery': 63,
         'redshift': 63,
         'scala': 62,
         'hadoop': 60,
         'git': 60,
         'github': 59,
         'cognos': 58,
         'dax': 56,
         'jira': 54,
         'nosql': 54,
         'numpy': 52,
    

In [60]:
entry_cleaned.description_tokens.iloc[14]

['excel', 'sas', 'spss', 'spreadsheet']

In [64]:
i = 14

print("OLD:", entry_cleaned.loc[i, "description_tokens"])
print("NEW:", tech_skills.iloc[i])

OLD: ['excel', 'sas', 'spss', 'spreadsheet']
NEW: ['excel', 'spss']


In [65]:
entry_cleaned.description.iloc[14]

'A pioneer in K-12 education since 2000, Amplify is leading the way in next-generation curriculum and assessment. Our core and supplemental programs in ELA, math, and science engage all students in rigorous learning and inspire them to think deeply, creatively, and for themselves. Our formative assessment products help teachers identify the targeted instruction students need to build a strong... foundation in early reading and math. All of our programs provide educators with powerful tools that help them understand and respond to the needs of every student. Today, Amplify serves more than 10 million students in all 50 states. For more information, visitamplify.com.\n\nAmplify seeks a hard-working individual to join our organization as an Associate Data Analyst. To do well in this role, you need an excellent eye for detail, experience as a data technician, and a deep understanding of popular data analysis tools and databases. Amplify offers many opportunities for professional growth to 

________________________________________

In [ ]:
# df2["is_entry_level"] = (
#     df2["Job Title"].str.lower().str.contains(title_pattern, regex=True, na=False)
#     |
#     df2["Job Description"].str.lower().str.contains(desc_pattern, regex=True, na=False)
# )

# df2["is_entry_level"] = df2["is_entry_level"] & (
#     ~df2["Job Title"].str.lower().str.contains(exclude_pattern, regex=True, na=False)
# )

In [ ]:
#entry_jobs2 = df2[df2["is_entry_level"]]

In [ ]:
#entry_jobs2

In [ ]:
# test[["salary_min_annual", "salary_max_annual", "salary_type"]] = df.apply(
#     lambda r: parse_salary_row(r["salary_pay"], r["salary_rate"]),
#     axis=1
# )
